<h1>MACORA's Global Manufacturing Performance Analysis </h1>

## Analysis Overview

In [76]:
import pandas as pd

# Convert json data to pandas dataframe
daikibo_df = pd.read_json("daikibo-telemetry-data.json")
daikibo_df.head(3)

,deviceID,deviceType,timestamp,location,data
0,19ff3161-2b3a-40a3-8604-bdc6532d0dab,CNC,2021-04-30 21:00:00,"{'country': 'japan', 'city': 'tokyo', 'area': ...","{'status': 'healthy', 'temperature': 27}"
1,19ff3161-2b3a-40a3-8604-bdc6532d0dab,CNC,2021-04-30 21:10:00,"{'country': 'japan', 'city': 'tokyo', 'area': ...","{'status': 'healthy', 'temperature': 27}"
2,19ff3161-2b3a-40a3-8604-bdc6532d0dab,CNC,2021-04-30 21:20:00,"{'country': 'japan', 'city': 'tokyo', 'area': ...","{'status': 'healthy', 'temperature': 26}"


In [77]:
# Check the df's dimensionality
daikibo_df.shape

(160704, 5)

In [78]:
# Gather column names, their datatype and non-null count
daikibo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160704 entries, 0 to 160703
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   deviceID    160704 non-null  object        
 1   deviceType  160704 non-null  object        
 2   timestamp   160704 non-null  datetime64[ns]
 3   location    160704 non-null  object        
 4   data        160704 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 6.1+ MB


In [79]:
# Identify the number of unique Device IDs
daikibo_df['deviceID'].nunique()

36

In [80]:
# Identify the number of unique Device types
daikibo_df['deviceType'].nunique()

9

In [81]:
# Identify the unique Device types
daikibo_df['deviceType'].unique()

array(['CNC', 'LaserCutter', 'HeavyDutyDrill', 'SpotWelder',
       'LaserWelder', 'MetalPress', 'Furnace', 'ConveyorBelt',
       'AirWrench'], dtype=object)

In [82]:
# Understand the structure of Location
daikibo_df['location'][0]

{'country': 'japan',
 'city': 'tokyo',
 'area': 'keiyō-industrial-zone',
 'factory': 'daikibo-factory-meiyo',
 'section': 'section-1'}

In [83]:
# Extract unique values for all location attributes
def unique_sublocations():
  location_keys = list(daikibo_df['location'].iloc[0].keys())
  for key in location_keys:
    unique_subloc = daikibo_df['location'].apply(lambda x: x[key]).unique()
    print(f"{key}:{unique_subloc}\n")

unique_sublocations()

country:['japan' 'germany' 'china']

city:['tokyo' 'osaka' 'berlin' 'shenzhen']

area:['keiyō-industrial-zone' 'keihanshin-industrial-zone'
 'adlershof-technology-park' 'jiaotang']

factory:['daikibo-factory-meiyo' 'daikibo-factory-seiko' 'daikibo-berlin'
 'daikibo-shenzhen']

section:['section-1' 'section-3' 'section-4' 'section-2']



In [102]:
# Add analysis overview summary
print("MACORA INDUSTRIES - Manufacturing Analytics")
print("="*43)
print(f"Analysis Period: {daikibo_df['timestamp'].min()} to {daikibo_df['timestamp'].max()}")
print(f"Total Records: {len(daikibo_df):,}")
print(f"Global Facilities: {daikibo_df['factory'].nunique()}")
print(f"Equipment Types: {daikibo_df['deviceType'].nunique()}")
print(f"Overall Health Rate: {(daikibo_df['status']=='healthy').mean()*100:.1f}%")

MACORA INDUSTRIES - Manufacturing Analytics
Analysis Period: 2021-04-30 21:00:00 to 2021-05-09 20:50:00
Total Records: 160,704
Global Facilities: 4
Equipment Types: 9
Overall Health Rate: 99.9%


## Identifying Issues

Based on the above overview, I identified a few questions to answer:

- What temperature range indicates healthy vs problematic operation?

- Which 20% of devices cause 80% of the failures?('unhealthy' status)

- Which factories outperform others and why?

- Which hours show disproportionate failure rates?('unhealthy' status)

- Ranking all device types from most to least reliable

## Feature Engineering

In [84]:
# Extract key variables for analysis
daikibo_df['hour'] = daikibo_df['timestamp'].dt.hour
daikibo_df['factory'] = daikibo_df['location'].apply(lambda x: x['factory'])
daikibo_df['temperature'] = daikibo_df['data'].apply(lambda x: x['temperature'])
daikibo_df['status'] = daikibo_df['data'].apply(lambda x: x['status'])

In [85]:
# Verify feature engineering results
daikibo_df.head(3)

,deviceID,deviceType,timestamp,location,data,hour,factory,temperature,status
0,19ff3161-2b3a-40a3-8604-bdc6532d0dab,CNC,2021-04-30 21:00:00,"{'country': 'japan', 'city': 'tokyo', 'area': ...","{'status': 'healthy', 'temperature': 27}",21,daikibo-factory-meiyo,27,healthy
1,19ff3161-2b3a-40a3-8604-bdc6532d0dab,CNC,2021-04-30 21:10:00,"{'country': 'japan', 'city': 'tokyo', 'area': ...","{'status': 'healthy', 'temperature': 27}",21,daikibo-factory-meiyo,27,healthy
2,19ff3161-2b3a-40a3-8604-bdc6532d0dab,CNC,2021-04-30 21:20:00,"{'country': 'japan', 'city': 'tokyo', 'area': ...","{'status': 'healthy', 'temperature': 26}",21,daikibo-factory-meiyo,26,healthy


## 1. What temperature range indicates healthy vs problematic operation?

In [86]:
# Fetch temperature's average by status
temp_by_status = daikibo_df.groupby('status')['temperature'].mean()
print(temp_by_status)

status
healthy      24.989981
unhealthy    24.883495
Name: temperature, dtype: float64


Mean stays consistant despite the status.
Let's identify the range of temperature by status, to find out the correlation between the both

In [87]:
# Fetch min and max temperature by status
temp_ranges = daikibo_df.groupby('status')['temperature'].agg(['min', 'max'])
print(temp_ranges)

           min  max
status             
healthy     23   27
unhealthy   23   27


**Conclusion:** Temperature is not correlated with machine failures

## 2. Which 20% of devices cause 80% of the failures?

In [88]:
# Identify the devices reported 'unhealthy' by deviceType(in %)
device_failures = daikibo_df.groupby('deviceType')['status'].apply(lambda x: (x == 'unhealthy').mean() * 100)
print(device_failures)

deviceType
AirWrench         0.000000
CNC               0.005600
ConveyorBelt      0.005600
Furnace           0.011201
HeavyDutyDrill    0.039203
LaserCutter       0.240815
LaserWelder       0.268817
MetalPress        0.000000
SpotWelder        0.005600
Name: status, dtype: float64


In [89]:
# Sort the devices reported 'unhealthy' in descending order
device_failures_sorted = device_failures.sort_values(ascending=False)

# Calculate the total devices reported 'unhealthy'
total_failure_rate = device_failures.sum()

# Create executive summary
top_2 = device_failures_sorted.head(2)
top_2_contribution = (top_2.sum() / total_failure_rate * 100)

print(f"\n📊 EXECUTIVE SUMMARY:")
print(f"• {list(top_2.index)} = {top_2_contribution:.1f}% of all failures")
print(f"• These 2 device types (out of {len(device_failures)}) represent {2/len(device_failures)*100:.0f}% of equipment")
print(f"• Classic 80/20 rule: {2/len(device_failures)*100:.0f}% of device types cause {top_2_contribution:.0f}% of problems")
print(f"• Focus maintenance resources on Laser equipment for maximum impact")


📊 EXECUTIVE SUMMARY:
• ['LaserWelder', 'LaserCutter'] = 88.3% of all failures
• These 2 device types (out of 9) represent 22% of equipment
• Classic 80/20 rule: 22% of device types cause 88% of problems
• Focus maintenance resources on Laser equipment for maximum impact


## 3. Which factories outperform others and why?

In [90]:
# Sort the devices reported 'unhealthy' in ascending order to identify the factory with best performance
daikibo_df.groupby(daikibo_df['factory'])['status'].apply(lambda x: (x == 'unhealthy').sum()).sort_values()

,status
factory,
daikibo-berlin,2
daikibo-factory-meiyo,11
daikibo-shenzhen,42
daikibo-factory-seiko,48


**Conclusion:** Berlin factory demonstrates best operational performance.

## 4. Which hours show disproportionate failure rates?

In [91]:
# Identify top 3 hours of the day, when device was reported 'unhealthy'
daikibo_df.groupby(daikibo_df['hour'])['status'].apply(lambda x: (x == 'unhealthy').sum()).sort_values(ascending=False).head(3)

,status
hour,
7,12
10,9
8,8


**Conclusion:** Could indicate one or more of the below:

- Cold start issues after overnight shutdown

- Morning shift operational problems

- Equipment stress during daily startup procedures

- Possible maintenance scheduling opportunities

## 5. Ranking all device types from most to least reliable

In [92]:
# Sort the deviceType in descending order when their status showed 'healthy' --> using SUM
daikibo_df.groupby(daikibo_df['deviceType'])['status'].apply(lambda x: (x == 'healthy').sum()).sort_values(ascending=False)

,status
deviceType,
AirWrench,17856
MetalPress,17856
CNC,17855
SpotWelder,17855
ConveyorBelt,17855
Furnace,17854
HeavyDutyDrill,17849
LaserCutter,17813
LaserWelder,17808


In [93]:
# Rechecking if each deviceType contains same number of records --> using COUNT
daikibo_df.groupby(daikibo_df['deviceType'])['status'].apply(lambda x: (x == 'healthy').count()).sort_values(ascending=False)

,status
deviceType,
AirWrench,17856
CNC,17856
ConveyorBelt,17856
Furnace,17856
HeavyDutyDrill,17856
LaserCutter,17856
LaserWelder,17856
MetalPress,17856
SpotWelder,17856


**Conclusion:** AirWrench and MetalPress are the most reliable equipment (zero or minimal failures)

##  Overall Executive Summary

In [101]:
print("="*38)
print("MACORA INDUSTRIES - EXECUTIVE SUMMARY")
print("="*38)
print()

# Analysis Overview
print("ANALYSIS SCOPE:")
print(f"- Data Period: {daikibo_df['timestamp'].min().strftime('%B %d')} - {daikibo_df['timestamp'].max().strftime('%B %d, %Y')}")
print(f"- Total Records: {len(daikibo_df):,}")
print(f"- Global Facilities: {daikibo_df['factory'].nunique()}")
print(f"- Equipment Types: {daikibo_df['deviceType'].nunique()}")
print(f"- Overall Equipment Health Rate: {(daikibo_df['status']=='healthy').mean()*100:.1f}%")
print()

# Key Findings Summary
print("KEY FINDINGS & RECOMMENDATIONS:")
print()

# 1: Temperature Analysis
print("1. TEMPERATURE IS NOT A FAILURE PREDICTOR")
temp_healthy = daikibo_df[daikibo_df['status']=='healthy']['temperature'].mean()
temp_unhealthy = daikibo_df[daikibo_df['status']=='unhealthy']['temperature'].mean()
print(f"- Healthy equipment avg: {temp_healthy:.1f}°C")
print(f"- Unhealthy equipment avg: {temp_unhealthy:.1f}°C")
print("- Recommendation: Focus maintenance resources on other factors")
print()

# 2: 80/20 Analysis
top_2_devices = device_failures_sorted.head(2)
top_2_contribution = (top_2_devices.sum() / device_failures.sum() * 100)
print("2. LASER EQUIPMENT DRIVES MAJORITY OF FAILURES")
print(f"- LaserWelder & LaserCutter: {top_2_contribution:.0f}% of all failures")
print(f"- Classic 80/20 principle confirmed")
print("- Recommendation: Prioritize laser equipment maintenance programs")
print()

# 3: Geographic Performance
best_factory = daikibo_df.groupby('factory')['status'].apply(lambda x: (x=='unhealthy').sum()).sort_values().index[0]
worst_factory = daikibo_df.groupby('factory')['status'].apply(lambda x: (x=='unhealthy').sum()).sort_values().index[-1]
best_failures = daikibo_df.groupby('factory')['status'].apply(lambda x: (x=='unhealthy').sum()).min()
worst_failures = daikibo_df.groupby('factory')['status'].apply(lambda x: (x=='unhealthy').sum()).max()

print("3. SIGNIFICANT GEOGRAPHIC PERFORMANCE GAPS")
print(f"- Best performing: {best_factory} ({best_failures} failures)")
print(f"- Worst performing: {worst_factory} ({worst_failures} failures)")
print(f"- Performance gap: {worst_failures/best_failures:.0f}x difference")
print("- Recommendation: Transfer Berlin best practices to other facilities")
print()

# 4: Temporal Patterns
peak_hour = daikibo_df.groupby('hour')['status'].apply(lambda x: (x=='unhealthy').sum()).sort_values(ascending=False).index[0]
peak_failures = daikibo_df.groupby('hour')['status'].apply(lambda x: (x=='unhealthy').sum()).max()
print("4. MORNING STARTUP VULNERABILITY IDENTIFIED")
print(f"- Peak failure time: {peak_hour}:00 AM ({peak_failures} failures)")
print("- Pattern suggests cold-start operational issues")
print("- Recommendation: Implement enhanced morning startup procedures")
print()

# 5: Equipment Reliability
most_reliable = daikibo_df.groupby('deviceType')['status'].apply(lambda x: (x=='healthy').sum()).sort_values(ascending=False).index[0]
least_reliable = daikibo_df.groupby('deviceType')['status'].apply(lambda x: (x=='healthy').sum()).sort_values(ascending=False).index[-1]
print("5. EQUIPMENT RELIABILITY HIERARCHY ESTABLISHED")
print(f"- Most reliable: {most_reliable}")
print(f"- Least reliable: {least_reliable}")
print("- Recommendation: Consider equipment replacement/upgrade priorities")
print()

# Business Impact
print("BUSINESS IMPACT:")
total_failures = (daikibo_df['status']=='unhealthy').sum()
failure_rate = (daikibo_df['status']=='unhealthy').mean() * 100
print(f"- Current failure rate: {failure_rate:.2f}%")
print(f"- Total failure events analyzed: {total_failures}")
print("- Identified specific improvement opportunities across operations")
print()

print("="*70)

MACORA INDUSTRIES - EXECUTIVE SUMMARY

ANALYSIS SCOPE:
• Data Period: April 30 - May 09, 2021
• Total Records: 160,704
• Global Facilities: 4
• Equipment Types: 9
• Overall Equipment Health Rate: 99.9%

KEY FINDINGS & RECOMMENDATIONS:

1. TEMPERATURE IS NOT A FAILURE PREDICTOR
- Healthy equipment avg: 25.0°C
- Unhealthy equipment avg: 24.9°C
- Recommendation: Focus maintenance resources on other factors

2. LASER EQUIPMENT DRIVES MAJORITY OF FAILURES
- LaserWelder & LaserCutter: 88% of all failures
- Classic 80/20 principle confirmed
- Recommendation: Prioritize laser equipment maintenance programs

3. SIGNIFICANT GEOGRAPHIC PERFORMANCE GAPS
- Best performing: daikibo-berlin (2 failures)
- Worst performing: daikibo-factory-seiko (48 failures)
- Performance gap: 24x difference
- Recommendation: Transfer Berlin best practices to other facilities

4. MORNING STARTUP VULNERABILITY IDENTIFIED
- Peak failure time: 7:00 AM (12 failures)
- Pattern suggests cold-start operational issues
- Recom